In [ ]:
#Date: Dec 11, 2024
#Author: Sonal Allana
#Purpose: To generate XAI metrics (faithfulness) for each explanation type

In [1]:
import numpy as np
from numpy import loadtxt
from numpy import savetxt
import tensorflow as tf
import os
import miattack
import xaimetrics

2024-10-29 15:07:38.961261: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Options (1) baseline_nn (2) dp_nn (3) syn_nn
base_folder = "baseline_nn"

if base_folder == "dp_nn":
    #Options (1) nm4000 (2) nm500 (3) nm66 (4) nm15
    nm_folder = "nm15"   #set the correct noise multiplier
elif base_folder == "syn_nn":
    #Options (1) ctgan (2) gausscopula (3) tvae 
    syndataType = "ctgan"
    

In [3]:
#Options (1) adult (2) credit (3) compas (4) hospital
dataset_name = "hospital"


In [4]:
#Uncomment for integrated gradient
#expl = "integrated_gradients"
#expl_folder = "ig"
#end integrated gradient

#Uncomment for smoothgrad
#expl = "smoothgrad"
#expl_folder = "sg"
#end smoothgrad

#Uncomment for shap
expl = "shap"
expl_folder = "shap"
#end shap

#Uncomment for lime
#expl = "lime"
#expl_folder = "lime"
#end lime

In [5]:
bpostmodel = True #Options (1) True (2) False (False for pre-model and in-model)
postmodelType = "dp_laplace" #Options (1) rand_laplace (2) rand_gaussian (3) dp_laplace (4) dp_gaussian (applicable for only post-model)


In [6]:
for i in range(3,4):
    #Load train-test files
    if base_folder == "dp_nn": #account for noise multiplier folder
        basepath = "../models/{0}/{1}/{2}/iter{3}/".format(base_folder,dataset_name,nm_folder,i)
        explpath = "../models/{0}/{1}/{2}/iter{3}/".format(base_folder,dataset_name,nm_folder,i)
        #For inmodel, explanations are compared with baseline (Non-DP) model 
        modelpath = "../models/baseline_nn/{0}/iter{1}/".format(dataset_name, i)
    elif base_folder == "syn_nn": #account for syn type folder
        basepath = "../models/{0}/{1}/{2}/iter{3}/".format(base_folder,dataset_name,syndataType,i)
        explpath = "../models/{0}/{1}/{2}/iter{3}/".format(base_folder,dataset_name,syndataType,i)
        #For premodel, explanations are compared with baseline (Non-DP) model 
        modelpath = "../models/baseline_nn/{0}/iter{1}/".format(dataset_name, i)
    else:
        basepath = "../models/{0}/{1}/iter{2}/".format(base_folder,dataset_name, i)
        explpath = "../models/{0}/{1}/iter{2}/".format(base_folder,dataset_name,i)
        modelpath = basepath
    
    fmodel = "model_wodp_iter{0}.keras".format(i)
    model = tf.keras.models.load_model(modelpath + fmodel,compile=False)
    model.summary()

    X_adv_train = loadtxt(explpath + expl_folder + '/X_adv_train.csv',delimiter=',')
    X_adv_test = loadtxt(explpath + expl_folder + '/X_adv_test.csv',delimiter=',')
    Y_adv_train = loadtxt(explpath + expl_folder + '/Y_adv_train.csv',delimiter=',')
    Y_adv_test = loadtxt(explpath + expl_folder + '/Y_adv_test.csv',delimiter=',')
    Z_adv_train = loadtxt(explpath + expl_folder + '/Z_adv_train.csv',delimiter=',')
    Z_adv_test = loadtxt(explpath + expl_folder + '/Z_adv_test.csv',delimiter=',')
    
    if not bpostmodel:
        attributions_train = loadtxt(explpath + expl_folder + '/attributions_train.csv',delimiter=',')
        attributions_test = loadtxt(explpath + expl_folder + '/attributions_test.csv',delimiter=',')
    else:
        expl_train_fname = '/attrs_train_{0}.csv'.format(postmodelType)
        expl_test_fname = '/attrs_test_{0}.csv'.format(postmodelType)
        attributions_train = loadtxt(explpath + expl_folder + expl_train_fname,delimiter=',')
        attributions_test = loadtxt(explpath + expl_folder + expl_test_fname,delimiter=',')
    
    print("Iteration ",i,":")
    samples = xaimetrics.getSamples(X_adv_test)
    x_batch_samples = X_adv_test[samples, :]
    y_batch_samples = Y_adv_test[samples]
    a_batch_samples = attributions_test[samples, :]
   
    xaiobj = xaimetrics.xai_metrics()
    xaiobj.calculateXaiMetrics(model,x_batch_samples,y_batch_samples,a_batch_samples)
    

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 120)]             0         
                                                                 
 dense1 (Dense)              (None, 1024)              123904    
                                                                 
 dense2 (Dense)              (None, 512)               524800    
                                                                 
 dense3 (Dense)              (None, 256)               131328    
                                                                 
 dense4 (Dense)              (None, 100)               25700     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 805833 (3.07 MB)
Trainable params: 805833 (3.

  0%|          | 0/2.0 [00:00<?, ?it/s]

Faithfulness Correlation - aggregate score:  [-0.007900072997766502]


  0%|          | 0/2.0 [00:00<?, ?it/s]

Faithfulness Estimate - aggregate score:  [0.007453125558784804]


  0%|          | 0/2.0 [00:00<?, ?it/s]

Sufficiency - aggregate score:  [0.328125]
